Agreguen texto para ver si queda en el mio

In [ ]:
import pandas as pd
archivo_excel = '/content/MP Consumos Barbosa.xlsx'
# hojas = pd.ExcelFile(archivo_excel).sheet_names
# hojas
sample_dfSoda = pd.read_excel(archivo_excel, sheet_name="Soda",decimal='.', header=0, usecols="C:AO", skiprows=2)

sample_dfSoda['TimeStamp'] = pd.to_datetime(sample_dfSoda['TimeStamp'])
sample_dfSoda.set_index('TimeStamp', inplace=True)
sample_dfSoda


,Partida R131,Estado Bombeo Semi R131,SP Semi R131,Estado Bombeo Receta R131,SP Receta R131,Consumo R131,Partida TK131,Estado Bombeo Semi TK131,SP Semi TK131,Consumo TK131,...,SP Semi R221,Estado Bombeo Receta R221,SP Receta R221,Consumo R221,Partida TK221,Estado Bombeo Semi TK221,SP Semi TK221,Estado Bombeo Receta TK221,SP Receta TK221,Consumo TK221
TimeStamp,,,,,,,,,,,,,,,,,,,,,
2024-10-22 16:40:29.032,10126959,Inactive,194.916870,Inactive,1096.303711,1287.890869,12345678,Active,638.449890,1788.473145,...,600,No Data,0,1959.967773,12345678,Inactive,223.772156,Inactive,0,2314.985596
2024-10-22 16:41:29.032,10126959,Inactive,194.898575,Inactive,1096.270264,1287.890869,12345678,Active,638.238098,1804.062256,...,600,No Data,0,1959.967773,12345678,Inactive,223.886414,Inactive,0,2314.985596
2024-10-22 16:42:29.032,10126959,Inactive,194.880264,Inactive,1096.236938,1287.890869,12345678,Active,638.026306,1819.701782,...,600,No Data,0,1959.967773,12345678,Inactive,224.000687,Inactive,0,2314.985596
2024-10-22 16:43:29.032,10126959,Inactive,194.861969,Inactive,1096.203613,1287.890869,12345678,Active,637.814453,1835.358887,...,600,No Data,0,1959.967773,12345678,Inactive,224.11496,Inactive,0,2314.985596
2024-10-22 16:44:29.032,10126959,Inactive,194.843674,Inactive,1096.170288,1287.890869,12345678,Active,637.602661,1850.113892,...,600,No Data,0,1959.967773,12345678,Inactive,224.229218,Inactive,0,2314.985596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-29 16:36:29.032,10127420,Inactive,12.000000,Inactive,793.000000,793.013672,10127430,Inactive,1030.000000,1029.105591,...,600,Inactive,0,1959.967773,12345678,Inactive,295,Inactive,0,295.028961
2024-10-29 16:37:29.032,10127420,Inactive,12.000000,Inactive,793.000000,793.013672,10127430,Inactive,1030.000000,1029.105591,...,600,Inactive,0,1959.967773,12345678,Inactive,295,Inactive,0,295.028961
2024-10-29 16:38:29.032,10127420,Inactive,12.000000,Inactive,793.000000,793.013672,10127430,Inactive,1030.000000,1029.105591,...,600,Inactive,0,1959.967773,12345678,Inactive,295,Inactive,0,295.028961


In [ ]:
def identificar_sp_activo(row, equipo):
    if 'Activo_Receta' + equipo in row:
        if row['Activo_Receta' + equipo]:
            return 'SP Receta ' + equipo
        elif row['Activo_Semi' + equipo]:
            return 'SP Semi ' + equipo
        else:
            return 'Ninguno'
    elif 'Activo_Semi' + equipo in row:
        if row['Activo_Semi' + equipo]:
            return 'SP Semi ' + equipo
    return 'Ninguno'




In [ ]:
# Identificar si el bombeo está activo
def CalculoMP(equipo):
    sample_dfSoda['Activo_Semi' + equipo] = sample_dfSoda['Estado Bombeo Semi '+ equipo] == 'Active'

    if 'Estado Bombeo Receta ' + equipo in sample_dfSoda.columns:
        sample_dfSoda['Activo_Receta' + equipo] = sample_dfSoda['Estado Bombeo Receta '+ equipo] == 'Active'


    # Crear una columna que identifique el nuevo SP activo
    # Aplicar la función para identificar el SP activo en cada momento
    # sample_dfSoda['SP_Activo' + equipo] = sample_dfSoda.apply(identificar_sp_activo, axis=1)
    sample_dfSoda['SP_Activo' + equipo] = sample_dfSoda.apply(identificar_sp_activo, axis=1, args=(equipo,))


    # Crear una columna con el valor del SP activo en cada momento
    sample_dfSoda['Valor_SP_Activo'] = sample_dfSoda.apply(
        lambda row: row['SP Receta ' + equipo] if row['SP_Activo' + equipo] == 'SP Receta ' + equipo
        else (row['SP Semi ' + equipo] if row['SP_Activo' + equipo] == 'SP Semi ' + equipo else 0),
        axis=1
    )

    # Identificar cambios en el SP activo (cada vez que se activa uno nuevo)
    sample_dfSoda['Cambio_SP_Activo'] = sample_dfSoda['SP_Activo' + equipo].shift(1) != sample_dfSoda['SP_Activo' + equipo]
    sample_dfSoda['Consumo R131_1'] = sample_dfSoda['Consumo ' + equipo].shift(-1)

    # Crear un número de solicitud único para cada nuevo SP activo
    sample_dfSoda['Solicitud'] = sample_dfSoda['Cambio_SP_Activo'].cumsum()

    # Agrupar los sample_dfSoda por solicitud y obtener los valores finales de SP y PV
    resultadoR131 = sample_dfSoda[sample_dfSoda['SP_Activo' + equipo] != 'Ninguno'].groupby('Solicitud').agg(
        SP_Activo_Final=('SP_Activo' + equipo, 'last'),
        Valor_SP_Final=('Valor_SP_Activo', 'max'),  # Tomar el máximo de los valores de SP para cada carga
        PV_Final=('Consumo R131_1', 'last')  # Tomar el último PV del grupo
    ).reset_index()
    resultadoR131["Equipo"]=equipo
    return resultadoR131
    # # Sumar todos los valores para cada carga y agregar un total
    # suma_filas = resultadoR131[['Valor_SP_Final', 'PV_Final']].sum().to_frame().T
    # suma_filas['SP_Activo_Final'] = 'Total ' + equipo
    # suma_filas['Solicitud'] = ' '

    # # Concatenar la fila de suma al DataFrame original
    # resultado_con_sumaR131Soda = pd.concat([resultadoR131, suma_filas], ignore_index=True)
    # # resultado_con_sumaR131Soda=resultado

    # # Ordenar el resultado final para que sea más legible
    # resultado_con_sumaR131Soda = resultado_con_sumaR131Soda[['Solicitud', 'SP_Activo_Final', 'Valor_SP_Final', 'PV_Final']]
    # resultado_con_sumaR131Soda["Equipo"]="R131"
    # # Verificar el resultado
    # return resultado_con_sumaR131Soda


In [ ]:
resultadoR131=CalculoMP("R131")
resultadoTK131=CalculoMP("TK131")
resultadoR141=CalculoMP("TK141")
resultadoV142=CalculoMP("V142")
resultadoTk141=CalculoMP("TK141")
resultadoR221=CalculoMP("R221")
resultadoTK221=CalculoMP("TK221")

resultados=pd.concat([resultadoR131,
                      resultadoTK131,
                      resultadoR141,
                      resultadoV142,
                      resultadoTk141,
                      resultadoR221,
                      resultadoTK221])
resultados

,Solicitud,SP_Activo_Final,Valor_SP_Final,PV_Final,Equipo
0,2,SP Receta R131,1086.929810,1287.476318,R131
1,4,SP Receta R131,1070.750610,1287.889404,R131
2,6,SP Receta R131,1022.079712,1287.872681,R131
3,8,SP Receta R131,1005.200073,1287.883179,R131
4,10,SP Receta R131,988.587219,1287.870483,R131
5,12,SP Receta R131,970.806824,1287.890991,R131
6,14,SP Receta R131,810.816711,1287.893677,R131
7,16,SP Receta R131,793.000000,793.01593,R131
8,18,SP Receta R131,793.000000,793.013672,R131
0,1,SP Semi TK131,638.449890,2500.017334,TK131


In [ ]:
# prompt: pivot_table = resultados.pivot_table(values=['Valor_SP_Final', 'PV_Final'], index=['Equipo','Solicitud','SP_Activo_Final'], aggfunc='sum', margins=True, margins_name='Total')
# prefero algo asi pero que subtotalice cada grupo

pivot_table = pd.pivot_table(resultados, values=['Valor_SP_Final', 'PV_Final'], index=['Equipo', 'Solicitud', 'SP_Activo_Final'], aggfunc='sum', margins=True, margins_name='Total')

# Subtotalizar por cada grupo (Equipo)
pivot_table_with_subtotals = pivot_table.groupby(level=0).sum().add_prefix('Subtotal_')

# Concatenar las tablas para obtener el resultado deseado
result = pd.concat([pivot_table, pivot_table_with_subtotals])

result

NameError: name 'resultados' is not defined